# Pulsim Tutorial

This notebook demonstrates how to use Pulsim for circuit simulation, focusing on power electronics applications.

## Setup

First, make sure you have built the Python module:
```bash
make python
```

Then run this notebook from the `build/python` directory or add it to your Python path.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pulsim as sl

print(f"Pulsim version: {sl.__version__}")

Running cmake --build & --install in /Users/lgili/Documents/01 - Codes/01 - Github/pulsim-core/build/cp313-cp313-macosx_26_0_arm64
Pulsim version: 0.1.0


## 1. Simple RC Circuit

Let's start with a basic RC circuit to understand the API.

In [2]:
# Create a simple RC circuit
circuit = sl.Circuit()

# Add components: voltage source, resistor, capacitor
circuit.add_voltage_source("V1", "in", "0", 5.0)  # 5V DC
circuit.add_resistor("R1", "in", "out", 1000.0)   # 1k ohm
circuit.add_capacitor("C1", "out", "0", 1e-6, ic=0.0)  # 1uF, initial voltage = 0

# Validate the circuit
valid, error = circuit.validate()
print(f"Circuit valid: {valid}")
print(f"Nodes: {circuit.node_names()}")
print(f"Total variables: {circuit.total_variables()}")

Circuit valid: True
Nodes: ['in', 'out']
Total variables: 3


In [3]:
# Configure simulation
opts = sl.SimulationOptions()
opts.tstart = 0.0
opts.tstop = 5e-3    # 5ms (5 time constants for RC = 1ms)
opts.dt = 1e-6       # 1us initial timestep
opts.dtmax = 10e-6   # 10us max timestep
opts.use_ic = True   # Use initial conditions

# Run simulation
result = sl.simulate(circuit, opts)

print(f"Simulation status: {result.final_status}")
print(f"Total steps: {result.total_steps}")
print(f"Available signals: {result.signal_names}")

Simulation status: SolverStatus.Success
Total steps: 509
Available signals: ['V(in)', 'V(out)', 'I(V1)']


In [ ]:
# Plot results
fig, ax = plt.subplots(figsize=(10, 5))

time_ms = np.array(result.time) * 1000  # Convert to ms

# Get signals from result data
data = result.to_dict()
v_in = data["signals"]["V(in)"]
v_out = data["signals"]["V(out)"]

ax.plot(time_ms, v_in, label="V(in)", linewidth=2)
ax.plot(time_ms, v_out, label="V(out)", linewidth=2)

# Add theoretical curve
tau = 1e-3  # RC = 1k * 1uF = 1ms
v_theory = 5.0 * (1 - np.exp(-np.array(result.time) / tau))
ax.plot(time_ms, v_theory, '--', label="Theoretical", alpha=0.7)

ax.set_xlabel("Time (ms)")
ax.set_ylabel("Voltage (V)")
ax.set_title("RC Circuit Step Response")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Buck Converter Simulation

Now let's simulate a more interesting circuit: a buck converter for power electronics.

In [5]:
# Create buck converter using JSON netlist (supports PWM waveforms)
buck_netlist = '''
{
    "name": "Buck Converter",
    "components": [
        {"type": "voltage_source", "name": "Vdc", "npos": "vcc", "nneg": "0",
         "waveform": {"type": "dc", "value": 48.0}},
        {"type": "voltage_source", "name": "Vpwm", "npos": "ctrl", "nneg": "0",
         "waveform": {"type": "pwm", "v_off": 0, "v_on": 10,
                      "frequency": 20000, "duty": 0.5, "dead_time": 0}},
        {"type": "switch", "name": "S1", "n1": "vcc", "n2": "sw", "ctrl_pos": "ctrl", "ctrl_neg": "0",
         "ron": 0.01, "roff": 1e9, "vth": 5.0},
        {"type": "diode", "name": "D1", "anode": "0", "cathode": "sw", "ideal": true},
        {"type": "inductor", "name": "L1", "n1": "sw", "n2": "out", "value": "100u", "ic": 0},
        {"type": "capacitor", "name": "C1", "n1": "out", "n2": "0", "value": "100u", "ic": 0},
        {"type": "resistor", "name": "Rload", "n1": "out", "n2": "0", "value": 10.0}
    ]
}
'''

buck = sl.parse_netlist_string(buck_netlist)
valid, error = buck.validate()
print(f"Buck converter valid: {valid}")
print(f"Nodes: {buck.node_names()}")

Buck converter valid: True
Nodes: ['vcc', 'ctrl', 'sw', 'out']


In [ ]:
# Simulate buck converter
opts = sl.SimulationOptions()
opts.tstart = 0.0
opts.tstop = 2e-3     # 2ms (40 switching cycles)
opts.dt = 100e-9      # 100ns initial timestep
opts.dtmax = 1e-6     # 1us max timestep
opts.use_ic = True
opts.abstol = 1e-9
opts.reltol = 1e-4

sim = sl.Simulator(buck, opts)
result = sim.run_transient()

print(f"Status: {result.final_status}")
print(f"Steps: {result.total_steps}")

In [ ]:
# Plot buck converter waveforms
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

time_us = np.array(result.time) * 1e6  # Convert to microseconds
data = result.to_dict()

# Plot 1: Control signal and switch node
ax1 = axes[0]
ax1.plot(time_us, data["signals"]["V(ctrl)"], label="V(ctrl)", alpha=0.8)
ax1.plot(time_us, data["signals"]["V(sw)"], label="V(sw)", alpha=0.8)
ax1.set_ylabel("Voltage (V)")
ax1.set_title("Buck Converter: Control and Switch Node")
ax1.legend(loc="upper right")
ax1.grid(True, alpha=0.3)

# Plot 2: Output voltage
ax2 = axes[1]
v_out = np.array(data["signals"]["V(out)"])
ax2.plot(time_us, v_out, label="V(out)", color="green", linewidth=1.5)
ax2.axhline(y=24.0, color='red', linestyle='--', alpha=0.5, label="Target (24V)")
ax2.set_ylabel("Voltage (V)")
ax2.set_title(f"Output Voltage (final: {v_out[-1]:.2f}V)")
ax2.legend(loc="lower right")
ax2.grid(True, alpha=0.3)

# Plot 3: Inductor current
ax3 = axes[2]
i_L = np.array(data["signals"]["I(L1)"])
ax3.plot(time_us, i_L, label="I(L1)", color="orange", linewidth=1.5)
ax3.set_xlabel("Time (μs)")
ax3.set_ylabel("Current (A)")
ax3.set_title(f"Inductor Current (final: {i_L[-1]:.2f}A)")
ax3.legend(loc="lower right")
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate power losses
losses = sim.power_losses()

print("Power Loss Summary:")
print(f"  Conduction losses: {losses.conduction_loss:.3f} W")
print(f"  Switching losses:  {losses.switching_loss:.3f} W")
print(f"  Total losses:      {losses.total_loss():.3f} W")

## 3. Zoom into Switching Waveforms

Let's look at a few switching cycles in detail.

In [ ]:
# Zoom into last few cycles
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Find indices for last 100us
t = np.array(result.time)
mask = t > (result.time[-1] - 100e-6)

time_zoom = t[mask] * 1e6
data = result.to_dict()

ax1 = axes[0]
v_sw = np.array(data["signals"]["V(sw)"])
ax1.plot(time_zoom, v_sw[mask], label="V(sw)", linewidth=1.5)
ax1.set_ylabel("Voltage (V)")
ax1.set_title("Switch Node Voltage (Zoomed)")
ax1.grid(True, alpha=0.3)
ax1.legend()

ax2 = axes[1]
i_L = np.array(data["signals"]["I(L1)"])
ax2.plot(time_zoom, i_L[mask], label="I(L1)", color="orange", linewidth=1.5)
ax2.set_xlabel("Time (μs)")
ax2.set_ylabel("Current (A)")
ax2.set_title("Inductor Current Ripple")
ax2.grid(True, alpha=0.3)
ax2.legend()

# Calculate ripple
i_ripple = i_L[mask].max() - i_L[mask].min()
print(f"Inductor current ripple: {i_ripple:.3f} A peak-to-peak")

plt.tight_layout()
plt.show()

## 4. Parsing Netlists from JSON

Pulsim can also load circuits from JSON netlist files.

In [ ]:
# Example: Parse a netlist from a JSON string
netlist_json = '''
{
    "name": "RLC Resonant Circuit",
    "components": [
        {"type": "voltage_source", "name": "V1", "npos": "in", "nneg": "0",
         "waveform": {"type": "pulse", "v1": 0, "v2": 1, "td": 0,
                      "tr": 1e-9, "tf": 1e-9, "pw": 1e-3, "period": 2e-3}},
        {"type": "resistor", "name": "R1", "n1": "in", "n2": "n1", "value": "10"},
        {"type": "inductor", "name": "L1", "n1": "n1", "n2": "out", "value": "100u"},
        {"type": "capacitor", "name": "C1", "n1": "out", "n2": "0", "value": "10n"}
    ]
}
'''

rlc = sl.parse_netlist_string(netlist_json)
print(f"Parsed circuit: {rlc.node_count()} nodes")

# Simulate
opts = sl.SimulationOptions()
opts.tstop = 500e-6
opts.dt = 10e-9
opts.dtmax = 100e-9

result = sl.simulate(rlc, opts)

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
time_us = np.array(result.time) * 1e6
data = result.to_dict()
ax.plot(time_us, data["signals"]["V(out)"], label="V(out)")
ax.set_xlabel("Time (μs)")
ax.set_ylabel("Voltage (V)")
ax.set_title("RLC Circuit Response")
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

# Calculate resonant frequency
L = 100e-6
C = 10e-9
f0 = 1 / (2 * np.pi * np.sqrt(L * C))
print(f"Theoretical resonant frequency: {f0/1000:.1f} kHz")

## 5. Thermal Simulation

Pulsim includes thermal modeling for power devices.

In [ ]:
# Create thermal model for a MOSFET
thermal_model = sl.create_mosfet_thermal(
    device_name="M1",
    rth_jc=0.5,   # Junction-to-case: 0.5 K/W
    rth_cs=0.3,   # Case-to-sink: 0.3 K/W
    rth_sa=2.0    # Sink-to-ambient: 2.0 K/W
)

print(f"Total Rth (junction to ambient): {thermal_model.rth_ja():.2f} K/W")
print(f"Foster network stages: {len(thermal_model.foster.stages)}")

# Create thermal simulator
thermal_sim = sl.ThermalSimulator()
thermal_sim.add_model(thermal_model)
thermal_sim.set_ambient(25.0)  # 25°C ambient
thermal_sim.initialize()

print(f"Initial junction temperature: {thermal_sim.junction_temp('M1'):.1f}°C")

In [ ]:
# Simulate thermal response to power dissipation
power = 10.0  # 10W dissipation
dt = 1e-3     # 1ms timestep
t_total = 5.0 # 5 seconds

times = []
temps = []

t = 0.0
while t < t_total:
    thermal_sim.step(dt, {"M1": power})
    times.append(t)
    temps.append(thermal_sim.junction_temp("M1"))
    t += dt

# Plot thermal response
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(times, temps, linewidth=2)

# Theoretical steady-state
t_ss = 25.0 + power * thermal_model.rth_ja()
ax.axhline(y=t_ss, color='red', linestyle='--', alpha=0.7, 
           label=f"Steady-state: {t_ss:.1f}°C")

ax.set_xlabel("Time (s)")
ax.set_ylabel("Junction Temperature (°C)")
ax.set_title(f"MOSFET Thermal Response (P = {power}W)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final junction temperature: {temps[-1]:.1f}°C")

## 6. Exporting Results

Simulation results can be easily exported for further analysis.

In [ ]:
# Convert results to dictionary (useful for pandas/JSON export)
data = result.to_dict()

print("Result dictionary keys:", list(data.keys()))
print(f"Number of time points: {len(data['time'])}")
print(f"Signals available: {list(data['signals'].keys())}")

In [ ]:
# Example: Create a pandas DataFrame
try:
    import pandas as pd
    
    df = pd.DataFrame({
        'time': data['time'],
        **data['signals']
    })
    
    print(df.head())
    print(f"\nDataFrame shape: {df.shape}")
    
    # Could save to CSV:
    # df.to_csv('simulation_results.csv', index=False)
except ImportError:
    print("pandas not installed - skipping DataFrame example")

## Summary

This tutorial covered:

1. **Basic circuit construction** - Creating circuits programmatically with resistors, capacitors, inductors, and voltage sources
2. **Transient simulation** - Running time-domain simulations with configurable options
3. **Power electronics** - Buck converter simulation with switches, diodes, and PWM control
4. **Netlist parsing** - Loading circuits from JSON format
5. **Thermal modeling** - MOSFET thermal simulation with Foster networks
6. **Data export** - Converting results for analysis with pandas/numpy

For more examples, check the `examples/` directory in the Pulsim repository.